In [8]:
import toml
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field

from rich import print as rprint

In [ ]:
llm_config = toml.load("../config.toml")["configs"][0]
llm = AzureChatOpenAI(
    api_version=llm_config["api_version"],
    azure_deployment=llm_config["deployment_name"],
    api_key=llm_config["api_key"],
    azure_endpoint=llm_config["api_endpoint"],
    model=llm_config["model_name"],
    temperature=0,
)
llm.invoke("hi")

In [5]:
class ExtractedInformation(BaseModel):
    subject: str = Field(description="The subject of the information")
    locations_found: list[str] = Field(description="The locations found in the text")
    people_found: list[str] = Field(description="The people found in the text")
    dates_found: list[str] = Field(description="The dates found in the text")
    numbers_found: list[str] = Field(description="The numbers found in the text")
    other_information: str = Field(
        description="The other information found in the text"
    )

In [ ]:
system_message = """
You are an expert at extracting information from text. You are extremely detailed oriented and will extract all information possiblefrom the text.
Respond in JSON format with no other commentary.

User: {query}. 
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_message,
        )
    ]
)

chain = prompt | llm.with_structured_output(ExtractedInformation)

response = chain.invoke(
    {
        "query": "New York City has a population of 8.4 million people. The weather is sunny and 70 degrees. Every day, a lot of people visit the city. They come from all over the world. Places like Spain, France, and Italy are popular. The city is known for its skyscrapers and museums. The city is also known for its pizza and bagels. The best time of year to visit is between April and June."
    }
)

rprint(response)